In [1]:
# Directory with raw data in csv
workingDir = '/scratch/paperscraper/csv/'

# Read in 111,420 records into dataframe with low_memory=False since we're not constrained by RAM
import pandas as pd
import codecs
papers_with_disciplines = pd.read_csv(workingDir + 'labeled_data.csv', low_memory=False, encoding='utf-8')
papers_with_disciplines = papers_with_disciplines[['abstract', 'classes']]

import unicodedata

# Used to sanitize abstracts
def sanitize_text(text):
    # if none, return empty characer
    if type(None) == type(text):
        return ''    
    # if not unicode, converting to string
    elif unicode != type(text):
        return str(text).lower()
    # else unicode, encoding to ascii after normalizing
    else:
        # NOTE: unicode character with no ascii equivilent are removed!
        return unicodedata.normalize('NFKD', text).encode('ascii','ignore').lower()

# Sanitize the abstracts
papers_with_disciplines['abstract'] = map(sanitize_text, papers_with_disciplines['abstract'])

In [2]:
# Make a binary indicator column for each class
import functools

# Split the class column and check for a specific class number
def isDiscipline(number, disciplines):
    if (str(number) in str(disciplines).split(',')):
        return 1
    else:
        return 0

classes = []
# Create a column for each class by performing a mapped search on the classes column
for i in range(0,6):
    papers_with_disciplines['class_' + str(i)] = map(functools.partial(isDiscipline, i), papers_with_disciplines['classes'])
    # Append class name to a list for future iteration
    classes.append('class_' + str(i))

In [3]:
import numpy as np

# Shuffle the data
shuffled = papers_with_disciplines.reindex(np.random.permutation(papers_with_disciplines.index))

# Randomly select 70% of data for training (hold-out 30%)
fraction_select = 0.70

# Assign 1s and 0s randomly
import random
def randomAssign(disciplines):
    if (random.random() <= fraction_select):
        return 1
    else:
        return 0
    
# Label random part of the data as training or not 
papers_with_disciplines['training'] = map(randomAssign, papers_with_disciplines['classes'])

# Split into a training dataframe and a testing dataframe
training_data = papers_with_disciplines[papers_with_disciplines['training'] == 1]
test_data = papers_with_disciplines[papers_with_disciplines['training'] == 0]

In [8]:
# Customize the count vectorizer by specifying a custom stopword vocabulary and stemmer

import nltk
import re

# Stopwords to remove
from nltk.corpus import stopwords
myStop = stopwords.words('english')

# French stop words for removal as well
myStop.extend(stopwords.words('french'))

# Remove duplicates across stop words
myStop = list(set(myStop))

# Stemmer did not perform better
# from nltk.stem.snowball import SnowballStemmer
# stemmer = SnowballStemmer('english')

# Custom tokenizer
class tokenize_custom(object):
    def __call__(self, doc):
        # First tokenize by sentence then by word
        tokens = [word for sent in nltk.sent_tokenize(doc) for word in nltk.word_tokenize(sent)]
        
        filtered_tokens = []
        for token in tokens:
            # Filter out any tokens not containing at least 2 letters (ex: numeric tokens, raw punctuation)
            if word not in myStop and len(re.sub('[^a-zA-Z]', '', word)) > 2:
                filtered_tokens.append(token)
                
        return filtered_tokens

In [5]:
# Import SciKitlearn libraries for the pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier

from pprint import pprint
import datetime
from time import time
import logging

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

In [6]:
from sklearn.metrics import matthews_corrcoef, make_scorer, f1_score, fbeta_score, classification_report, confusion_matrix

# ROC curve acts as a good visual indicator of performance
from sklearn.metrics import roc_curve, auc

import matplotlib.pyplot as plt
# Display figures inline
%matplotlib inline

# Create a tiny font for figure labelling
from matplotlib.font_manager import FontProperties
fontP = FontProperties()
fontP.set_size('small')

# Define a custom function which reports the performance of a model
def performance_metrics(probas, prediction, actual):
    ### Plot the ROC curve ###

    # Actual y outcomes vs. probabilities predicted by the final model
    fpr, tpr, thresholds = roc_curve(actual, probas[:, 1])
    
    # Create the ROC_AUC object using auc
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=1, label='(%0.2f)' % (roc_auc))

    # A single guess curve needs plotting (the curve TP=FP or y = x)
    plt.plot([0, 1], [0, 1], '--', color = (0.6, 0.6, 0.6), label = 'Guess Curve')

    # Set reasonable limits to x and y, based on the fact that FP, TP are [0, 1]
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    
    # Label the chart
    plt.title('Receiver operating characteristic for class '+classex)
    
    # Make a nice legend that is out of the plot
    plt.legend(loc='center left', bbox_to_anchor = (1, 0.5), prop = fontP)
              
    plt.show()
    ### Done ROC curve ###
    
    ### Confusion matrix ###
    cm = confusion_matrix(np.asarray(actual.tolist()), prediction)
    print (cm)
    print ('')
    
    ### MCC ###
    print 'MCC: ' + str(matthews_corrcoef(np.asarray(actual.tolist()), prediction))
    print ('')
    
    ### Report ###    
    print 'Report:'
    print classification_report(np.asarray(actual.tolist()), prediction)

In [9]:
# Number of folds for grid cross-validation; 10 was recommended across several resources
folds = 10

# Make an MCC scorer for models
scorer = make_scorer(matthews_corrcoef)

# Define a model estimation pipeline
pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer = tokenize_custom(), 
                                             stop_words = myStop, 
                                             min_df = 5,
                                             max_df = 0.4,
                                             ngram_range = (1,3)
                                             
        )
    ),
        
    ('clf', SGDClassifier(n_iter = 20, 
                                            penalty = 'elasticnet',
                                            loss = 'modified_huber'
         )
    )
])

# Hyperparameters to test; note that these grow combinatorially and will consume very large amounts of memory 
parameters = {
    'vect__sublinear_tf': (True, False),
    'clf__class_weight': ('balanced', None)
}

# Maximum cores in environment
import multiprocessing
max_usage = 0.5

max_cores = int(max_usage*multiprocessing.cpu_count())

# Figure out the worst case number of cores used by figuring out the number of parameter combinations
combinations = 1
for parameter_option_list in parameters.itervalues():
    combinations = combinations*len(parameter_option_list)
print (str(combinations) + " number of combinations to be tested")
    
# Number of cores to use
cores = int(max_cores/combinations)
print(str(cores) +" cores will be used, with a max of " + str(max_cores) + " at a time")

4 number of combinations to be tested
8 cores will be used, with a max of 32 at a time


In [ ]:
# List of the best estimators for each class
best_models = []

if __name__ == '__main__':
    for classex in classes:   
                
        # Find the best parameters for both the feature extraction and the classifier
        grid_search = GridSearchCV(pipeline, parameters, scoring = scorer, cv = folds,
                                  n_jobs = cores, pre_dispatch = max_cores, verbose = 0)

        print('Performing grid search on '+classex + ' --- ' + str(datetime.datetime.now().time()))               
        t0 = time()
        # Estimate classifier
        grid_search.fit(training_data['abstract'], training_data[classex])
        print('done in %0.3fs' % (time() - t0))

        print('Best model score: %0.3f' % grid_search.best_score_)        

        best_estimator = grid_search.best_estimator_
        best_models.append(best_estimator)
        
        if len(parameters) > 0:
            best_parameters = best_estimator.get_params()        
            print('Best parameters set:')
            for param_name in sorted(parameters.keys()):
                print('\t%s: %r' % (param_name, best_parameters[param_name])) 
            print ('')
        
        print('--------------------------------------------------------------------------')
        print('Training performance')
        # Probabilities, predictions, true values on training data
        performance_metrics(best_estimator.predict_proba(training_data['abstract']), best_estimator.predict(training_data['abstract']), training_data[classex])
        print ('')

        print('--------------------------------------------------------------------------')
        print('Hold-out performance')
        # Probabilities, predictions, true values on hold-out data
        performance_metrics(best_estimator.predict_proba(test_data['abstract']), best_estimator.predict(test_data['abstract']), test_data[classex])
        print ('')

        print('--------------------------------------------------------------------------')

Performing grid search on class_0 --- 01:37:51.426565
